# Data Wrangling - Load CSV
### POD Allocation | Project 3

Goal: Load CSV into relational databases

Steps

- Setup Cursors and connection to Containers
- Create Relational Databases
- Load CSV files into Relational Databases

In [1]:
import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
cursor = connection.cursor()

## Reading CSV Files

In [5]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [6]:
my_read_csv_file("assets.csv", limit=10)

['asset_id', 'asset_name', 'asset_class', 'country_of_risk', 'sec_type']
['MSI US Equity', 'Motorola Solutions Inc', 'EQUI', 'US', 'Common Stock']
['CMG US Equity', 'Chipotle Mexican Grill Inc', 'EQUI', 'US', 'Common Stock']
['TMO US Equity', 'Thermo Fisher Scientific Inc', 'EQUI', 'US', 'Common Stock']
['COST US Equity', 'Costco Wholesale Corp', 'EQUI', 'US', 'Common Stock']
['UPS US Equity', 'United Parcel Service Inc', 'EQUI', 'US', 'Common Stock']
['FANG US Equity', 'Diamondback Energy Inc', 'EQUI', 'US', 'Common Stock']
['ESM5 Index', 'S&P500 EMINI FUT  Jun25', 'EDRV', 'US', 'Future']
['GOOGL US Equity', 'Alphabet Inc', 'EQUI', 'US', 'Common Stock']
['VRTX US Equity', 'Vertex Pharmaceuticals Inc', 'EQUI', 'US', 'Common Stock']

Printed  10 lines of  3818 total lines.


## Loading CSV Data into Database Tables

I am planning to create the following tables:

- temp_pods
- temp_assets
- temp_pod_nav_history
- temp_asset_factor_exposures
- temp_pods_current_allocation

In [7]:
#
# drop all the temp tables in the foreign key order
#


connection.rollback()

query = """

drop table if exists temp_pods;
drop table if exists temp_assets;
drop table if exists temp_pod_nav_history;
drop table if exists temp_pod_ret_history;
drop table if exists temp_asset_factor_exposures;
drop table if exists temp_pods_current_allocation;


"""

cursor.execute(query)

connection.commit()


In [8]:
#
# create all the temp tables in the foreign key order
#

connection.rollback()

query = """

create table temp_pods (
  pod_id varchar(100),
  pod_name varchar(100),
  pod_asset_class_focus varchar(100),
  pod_geo_focus varchar(100),
  pod_inception_date varchar(100)
);

create table temp_assets (
  asset_id varchar(100),
  asset_name varchar(100),
  asset_class varchar(100),
  country_of_risk varchar(100),
  sec_type varchar(100)
);

create table temp_pod_nav_history (
  date date,
  pod_id varchar(100),
  price numeric
);

create table temp_pod_ret_history (
  date date,
  pod_id varchar(100),
  return numeric
);

create table temp_asset_factor_exposures (
  asset_id varchar(100),
  rate_coef decimal(10, 6),
  equity_coef decimal(10, 6),
  credit_coef decimal(10, 6),
  fx_coef decimal(10, 6),
  inflation_coef decimal(10, 6)
);

create table temp_pods_current_allocation (
  pod_id varchar(100),
  asset_id varchar(100),
  weight varchar(100)
);

"""

cursor.execute(query)

connection.commit()

#### pods.csv

In [9]:
# load the csv file into the database table

connection.rollback()

query = """

copy temp_pods
from '/user/projects/project-3-lelvieira-berkeley/code/pods.csv' delimiter ',' NULL '' csv header;


"""

cursor.execute(query)

connection.commit()

In [10]:
#
# query PODS to see if table was loaded successfully
#

rollback_before_flag = True
rollback_after_flag = True

query = """

select * 
from temp_pods
limit 5;

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,pod_id,pod_name,pod_asset_class_focus,pod_geo_focus,pod_inception_date
0,GSEMCBI LX,GS EMRG MRKT CORP BD PT IA,Fixed Income,Global,5/17/2011
1,AUDHYBI LX,AXA WF-US DYN HIGH YLD BD-I,Fixed Income,U.S.,1/15/2014
2,SPY US,SPDR S&P 500 ETF TRUST,Equity,U.S.,1/22/1993
3,WSEEAUH ID,WELL STRAT EUR EQ-S EUR AC,Equity,European Region,12/15/2011
4,QLEIX US,AQR LONG-SHORT EQUITY-I,Alternative,U.S.,7/16/2013


#### assets.csv

In [11]:
# load the csv file into the database table

connection.rollback()

query = """


copy temp_assets
from '/user/projects/project-3-lelvieira-berkeley/code/assets.csv' delimiter ',' NULL '' csv header;



"""

cursor.execute(query)

connection.commit()

In [12]:
#
# query ASSETS to see if table was loaded successfully
#

rollback_before_flag = True
rollback_after_flag = True

query = """

select * 
from temp_assets
limit 5;

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,asset_id,asset_name,asset_class,country_of_risk,sec_type
0,MSI US Equity,Motorola Solutions Inc,EQUI,US,Common Stock
1,CMG US Equity,Chipotle Mexican Grill Inc,EQUI,US,Common Stock
2,TMO US Equity,Thermo Fisher Scientific Inc,EQUI,US,Common Stock
3,COST US Equity,Costco Wholesale Corp,EQUI,US,Common Stock
4,UPS US Equity,United Parcel Service Inc,EQUI,US,Common Stock


#### pod_nav_history

In [13]:
# load the csv file into the database table

connection.rollback()

query = """


copy temp_pod_nav_history
from '/user/projects/project-3-lelvieira-berkeley/code/pod_nav_history.csv' delimiter ',' NULL '' csv header;


"""

cursor.execute(query)

connection.commit()

In [14]:
#
# query TEMP_POD_NAV_HISTORY to see if table was loaded successfully
#

rollback_before_flag = True
rollback_after_flag = True

query = """

select * 
from temp_pod_nav_history
limit 5;

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,date,pod_id,price
0,2014-12-05,0431214C JP,10046
1,2014-12-12,0431214C JP,9802
2,2014-12-19,0431214C JP,9826
3,2014-12-26,0431214C JP,10040
4,2015-01-02,0431214C JP,9976


#### asset_factor_exposures

In [15]:
# load the csv file into the database table

connection.rollback()

query = """


copy temp_asset_factor_exposures
from '/user/projects/project-3-lelvieira-berkeley/code/asset_factor_exposures.csv' delimiter ',' NULL '' csv header;


"""

cursor.execute(query)

connection.commit()

In [16]:
#
# query ASSET_FACTOR_EXPOSURES to see if table was loaded successfully
#

rollback_before_flag = True
rollback_after_flag = True

query = """

select * 
from temp_asset_factor_exposures
limit 5;

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,asset_id,rate_coef,equity_coef,credit_coef,fx_coef,inflation_coef
0,MSI US Equity,0.000000,1.257860,0.372712,0,-0.585354
1,CMG US Equity,1.275231,1.083837,0.000000,0,0.866178
2,TMO US Equity,0.000000,0.981911,0.000000,0,0.000000
3,COST US Equity,0.653002,0.763462,-1.280612,0,0.293192
4,UPS US Equity,0.000000,0.940138,-1.934936,0,0.000000


#### pods_current_allocation

In [17]:
# load the csv file into the database table

connection.rollback()

query = """


copy temp_pods_current_allocation
from '/user/projects/project-3-lelvieira-berkeley/code/pods_current_allocation.csv' delimiter ',' NULL '' csv header;


"""

cursor.execute(query)

connection.commit()

In [18]:
#
# query ASSET_FACTOR_EXPOSURES to see if table was loaded successfully
#

rollback_before_flag = True
rollback_after_flag = True

query = """

select * 
from temp_pods_current_allocation
limit 5;

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,pod_id,asset_id,weight
0,JHQAX US,MSI US Equity,0.0013960794015190992
1,JHQAX US,CMG US Equity,0.004976350583064751
2,JHQAX US,TMO US Equity,0.007356123558486676
3,JHQAX US,COST US Equity,0.0040654028942667805
4,JHQAX US,UPS US Equity,0.003498533603449861
